In [ ]:
from anndata import AnnData
import scanpy as sc

import matplotlib.pyplot as plt
import pandas as pd
from ALLCools.plot import *
from skimage.transform import rotate, SimilarityTransform, matrix_transform
import numpy as np
import seaborn as sns

In [ ]:
batch = 'b2-2'
model = 'watershed'
region = 'region_1'

In [ ]:
adata  = sc.read_h5ad(f'./adata/{model}_{region}.h5ad')
cell_by_gene = pd.read_csv(f'./qc_meta/{region}_{model}_afterQC_cell_by_gene.csv', 
                           index_col = 0).sort_index()
cell_meta = pd.read_csv(f'./qc_meta/{region}_{model}_afterQC_cell_meta.csv', 
                        index_col=0).sort_index()

In [ ]:
adata.obsm['X_spatial'] = adata.obsm['spatial'].copy()
del adata.obsm['spatial']

In [ ]:
print(f'cell number: {len(adata.obs_names)}')

# plot umap

In [ ]:
fig, axes = plt.subplots(figsize = (10,5), ncols = 2,dpi = 200)

ax = axes[0]
categorical_scatter(data=adata,
                        ax=ax,
                        coord_base='umap',
                        hue='leiden',
                        text_anno='leiden',
                        show_legend=True,
                       max_points = None)
ax.set(title = 'cluster')

ax = axes[1]
continuous_scatter(data=adata,
                        ax=ax,
                        coord_base='umap',
                        hue=cell_meta['volume'],
                       max_points = None)

ax.set(title = 'cell_volume')



## plot cluster spatial

In [ ]:
# rotate spatial
trans_mat = SimilarityTransform(rotation=6.7)
rotate_coord = matrix_transform(adata.obsm['X_spatial'], np.array(trans_mat))
adata.obsm['X_spatial_rotated'] = rotate_coord.copy()

In [ ]:
fig, axes = plt.subplots(figsize = (5,3), ncols = 2, dpi = 200)

ax = axes[0]
categorical_scatter(data=adata,
                        ax=ax,
                        coord_base='spatial_rotated',
                        hue='leiden',
                        #text_anno='leiden',
                        show_legend=False,
                       max_points = None)
ax.set(title = 'cluster')

ax = axes[1]
continuous_scatter(data=adata,
                        ax=ax,
                        coord_base='spatial_rotated',
                        hue=cell_meta['volume'],
                       max_points = None)

ax.set(title = 'cell_volume')


In [ ]:
cluster_list = list(adata.obs['leiden'].unique().sort_values())
n_plots = len(cluster_list)

fig, axes = plt.subplots(figsize=(12, 8),
                         nrows=4,
                         ncols=6,
                         dpi=300,
                         constrained_layout=True,
                         sharex=False)

for ax, cluster in zip(axes.ravel(), cluster_list):
    categorical_scatter(data=adata,
                            ax=ax,
                            coord_base='spatial_rotated',
                            hue=adata.obs['leiden'].isin([cluster]),
                            #text_anno='leiden',
                            show_legend=False,
                            palette = {
                                        True: 'red',
                                        False: 'lightgrey'
                                    },
                           max_points = None)
    ax.set( title = cluster) 
    
for ax in axes.ravel()[n_plots:]:
    ax.axis('off')

In [ ]:
adata.write_h5ad(f'./adata/{batch}_{model}_{region}_rotated.h5ad')

# plot marker gene

In [ ]:
# fish_genes = ['Fos','Snap25','Slc1a2','Slc17a7','Gad1','Mbp']
# fig, axes = plt.subplots(figsize = (12,4), ncols = 3, nrows =2, dpi = 200)

# for ax, gene in zip(axes.ravel(), fish_genes):
#     continuous_scatter(data=adata, 
#                    coord_base='spatial_rotated',
#                    ax = ax,
#                    hue=cell_by_gene[gene],
#                    max_points=None)
#     ax.set( title = gene) 
    
# for ax in axes.ravel()[n_plots:]:
#     ax.axis('off')

In [ ]:
# fig, ax = plt.subplots(figsize = (4,2), ncols = 1, nrows =1, dpi = 200)

# continuous_scatter(data=adata, 
#                    coord_base='spatial_rotated',
#                    ax = ax,
#                    hue=cell_by_gene['Tshz2'],
#                    max_points=None)
# ax.set( title = 'Tshz2') 

In [ ]:
# fig, axes = plt.subplots(figsize = (10,58), ncols = 5, nrows =29, dpi = 200)

# for ax, gene in zip(axes.ravel(), cef_list):
#     continuous_scatter(data=adata, 
#                    coord_base='umap',
#                    ax = ax,
#                    hue=cell_by_gene[gene],
#                    max_points=None)
#     ax.set( title = gene) 
